# <center>ytree: yt for merger trees</center>
### <center>Britton Smith, University of Edinburgh</center>
#### <center>yt user/developer workshop - July 17, 2025</center>

## <center>Halo Assembly as a Merger Tree</center>

<p>

<center><img src="ytree-header.png" alt="Drawing" style="height: 80%;width: 80%"/></center>

## ytree is yt for merger trees
* Amiga Halo Finder
* Consistent-Trees
* Consistent-Trees-HDF5
* Gadget4
* LHaloTree
* LHaloTree-HDF5
* MORIA
* Rockstar Catalogs
* TreeFarm
* TreeFrog
* Saved Arbors (ytree format)

## Using `ytree`

This should seem familiar.

In [ ]:
import ytree

In [ ]:
arbor = ytree.load("/Users/britton/EnzoRuns/ytree_data/consistent_trees/locations.dat")

In [ ]:
print (arbor.hubble_constant, arbor.omega_matter, arbor.omega_lambda, arbor.box_size)

In [ ]:
print (arbor.field_list)

### `Arbor`: a collection of trees (or so I was led to believe...)

All trees are accessed through the arbor.

In [ ]:
# How many trees in this dataset?
print(arbor.size)

### The arbor acts as a generator of trees

In [ ]:
i = arbor['mass'].argmax()
my_tree = arbor[i]

In [ ]:
print(my_tree, my_tree['mass'], my_tree['redshift'])

In [ ]:
all_trees = list(arbor[:])
print (all_trees[0]['virial_radius'])

### A tree is a collection of linked `TreeNode` objects

In [ ]:
my_tree = arbor[0]
print("Me:", my_tree)
ancestors = list(my_tree.ancestors)
print("My ancestors:", list(ancestors))
print("My ancestor's descendent (me):", ancestors[0].descendent)

### Accessing all members of a tree

The member halos of a tree can be generated from any `TreeNode`.
- `tree` - all halos in the tree
- `forest` - all halos in the forest
- `prog` - the main progenitors of the current halo

In [ ]:
print(f"Halos in this tree: {len(list(my_tree['tree']))}.")
print(f"Main progenitors of the tree: {len(list(my_tree['prog']))}.")

### Fields for a tree

In [ ]:
print(my_tree['prog', 'virial_radius'].to('Mpc/h'))

### Derived Fields

In [ ]:
import yt
from yt.utilities.physical_constants import G
def _potential_field(field, data):
    return data['mass'] / data['virial_radius'] * G

In [ ]:
arbor.add_derived_field("potential", _potential_field, 'erg/g')

In [ ]:
print(my_tree['prog', 'potential'])

### Analysis Fields

Field values are result of complex or external analysis.

In [ ]:
import numpy as np
arbor.add_analysis_field('teds', units='', dtype=int, default=-1)

In [ ]:
for halo in my_tree['tree']:
    halo['teds'] = np.random.randint(0, 10)

In [ ]:
print(my_tree['tree', 'teds'])

### Re-save the arbor to an optimized format

In [ ]:
fn = arbor.save_arbor()

In [ ]:
new_arbor = ytree.load(fn)

### Or save a single tree

In [ ]:
treefn = my_tree.save_tree()
new_tree = ytree.load(treefn)

In [ ]:
print (new_tree[0]['prog', 'teds'])

### Visualization

`TreePlot` makes customizable plots.

In [ ]:
p = ytree.TreePlot(my_tree, dot_kwargs={'rankdir': 'LR', 'size': '"12,4"'})
p.save('my_tree.png')

<center><img src="my_tree.png" alt="Drawing" style="height: 80%;width: 80%"/></center>

#### Customizing tree plots
Custom functions can be supplied defining the appearance of nodes and edges. See the docs!

## Loading halos with yt
* ytree format data can be loaded with `yt`

In [ ]:
ds = new_arbor.ytds

In [ ]:
sp = ds.sphere(ds.domain_center, 0.4*ds.domain_width[0])

In [ ]:
sp["halos", "mass"]

In [ ]:
p = yt.ParticleProjectionPlot(ds, "x", ("halos", "redshift"), data_source=sp)
p.set_cmap(("halos", "redshift"), "algae")

## Selecting halos with yt
* select halos according to criteria

In [ ]:
selection = new_arbor.get_yt_selection(
    below=[("mass", 1e13, "Msun")],
    above=[("redshift", 1)])

In [ ]:
selection["mass"].to("Msun").max()

## What else
* [parallelism](https://ytree.readthedocs.io/en/latest/Parallelism.html)
  * functions for parallelizing over trees, all halos in a single tree, a general selection of halos
* [AnalysisPipeline](https://ytree.readthedocs.io/en/latest/Analysis.html)
  * define a complex analysis workflow to save analysis fields
  * parallel-safe analysis and saving of new fields

## ytree Resources

* Installation: `pip install ytree` or `conda install ytree`
* Documentation: **ytree.readthedocs.io**
* Sample data: **"ytree" collection on yt hub**
* Source code: `https://github.com/ytree-project/ytree`